In [3]:
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from scipy.io import loadmat
from Laplacian import Laplacian

In [4]:
n_node = 10 # num of nodes in hidden layer
lam_r = 1 # RR regularization parameter
lam_l = 1 # MR regularization parameter
w_range = [-1, 1] # range of random weights
b_range = [0, 1] # range of random biases

class RVFL:
    """ RVFL Classifier """
    
    def __init__(self, n_node, C0, lam, w_range, b_range, activation='relu', same_feature=False):
        self.n_node = n_node
        self.C0 = C0
        self.lam = lam
        self.w_range = w_range
        self.b_range = b_range
        self.weight = None
        self.bias = None
        self.beta = None
        a = Activation()
        self.activation_function = getattr(a, activation)
        self.std = None
        self.mean = None
        self.same_feature = same_feature
        
    def train(self, data, label, n_class):
        assert len(data.shape) > 1
        assert len(data) == len(label)
        assert len(label.shape) == 1
        
        data = self.standardize(data) # Normalize
        n_sample = len(data)
        n_feature = len(data[0])
        self.weight = (self.w_range[1] - self.w_range[0]) * np.random.random([n_feature, self.n_node]) + self.w_range[0]
        self.bias = (self.b_range[1] - self.b_range[0]) * np.random.random([1, self.n_node]) + self.b_range[0]
        
        h = self.activation_function(np.dot(data, self.weight) + np.dot(np.ones([n_sample, 1]), self.bias))
        d = np.concatenate([h, data], axis=1)
        d = np.concatenate([d, np.ones_like(d[:, 0:1])], axis=1) # concat column of 1s
        y = self.one_hot_encoding(label, n_class)
        L = np.asarray(Laplacian(data=data, k=3))
        # Minimize training complexity
        self.beta = self.beta_function(d, L, self.C0, self.lam, y)
        print(self.beta)
        
    def beta_function(self, H, L, C0, lam, y):
        # number of samples per class label
        n_sample, n_feature = np.shape(H)
        C = np.identity(n_sample) * C0

        # More labeled examples than hidden neurons
        if n_sample > (self.n_node + n_feature):
            I = np.identity(n_feature)
            inv_arg = I + (H.T @ C @ H) + lam * (H.T @ L @ H)
            beta = np.linalg.inv(inv_arg) @ H.T @ C @ y

        # Less labeled examples than hidden neurons (apparently the more common case)
        else:
            I = np.identity(n_sample)
            inv_arg = I + (C @ H @ H.T) + lam * (L @ H @ H.T)
            beta = (H.T @ np.linalg.inv(inv_arg) @ C @ y)
            
        return beta

    def predict(self, data, raw_output=False):
        data = self.standardize(data) # Normalize
        h = self.activation_function(np.dot(data, self.weight) + self.bias)
        d = np.concatenate([h, data], axis=1)
        d = np.concatenate([d, np.ones_like(d[:, 0:1])], axis=1)
        result = self.softmax(np.dot(d, self.beta))
        if not raw_output:
            result = np.argmax(result, axis=1)
        return result
    
    def eval(self, data, label):
        assert len(data.shape) > 1
        assert len(data) == len(label)
        assert len(label.shape) == 1
        
        result = self.predict(data, False)
        acc = np.sum(np.equal(result, label))/len(label)
        return acc
        
    def one_hot_encoding(self, label, n_class):
        y = np.zeros([len(label), n_class])
        for i in range(len(label)):
            y[i, label[i]] = 1
        return y
    
    def standardize(self, x):
        if self.same_feature is True:
            if self.std is None:
                self.std = np.maximum(np.std(x), 1/np.sqrt(len(x)))
            if self.mean is None:
                self.mean = np.mean(x)
            return (x - self.mean) / self.std
        else:
            if self.std is None:
                self.std = np.maximum(np.std(x, axis=0), 1/np.sqrt(len(x)))
            if self.mean is None:
                self.mean = np.mean(x, axis=0)
            return (x - self.mean) / self.std
        
    def softmax(self, x):
        return np.exp(x) / np.repeat((np.sum(np.exp(x), axis=1))[:, np.newaxis], len(x[0]), axis=1)
        
class Activation:
    def sigmoid(self, x):
        return 1 / (1 + np.e ** (-x))
    
    def sine(self, x):
        return np.sin(x)
    
    def sign(self, x):
        return np.sign(x)
    
    
    def relu(self, x):
        return np.maximum(0, x)

In [5]:
# coil20 dataset
if __name__=="__main__":
    dataset = loadmat('coil20.mat')
    label = np.array([dataset['Y'][i][0] - 1 for i in range(len(dataset['Y']))])
    data = dataset['X']
    n_class = len(np.unique(label))

    # train-test-split
    X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42)
    kf = KFold(n_splits=10, shuffle=True, random_state=0)
    val_acc = []
    max_index = -1
    
    for i, kf_values in enumerate(kf.split(X_train, y_train)):
    #     print(f'train: {train_index}, val: {val_index}')
        print('Validation: {}'.format(i + 1))
        train_index, val_index = kf_values
        X_val_train, X_val_test = X_train[train_index], X_train[val_index]
        y_val_train, y_val_test = y_train[train_index], y_train[val_index]
        model = RVFL(n_node, lam_r, lam_l,  w_range, b_range)
        model.train(X_val_train, y_val_train, n_class)
        prediction = model.predict(X_val_test, True)
        acc = model.eval(X_val_test, y_val_test)
        print(f'Validation accuracy: {acc}')
        val_acc.append(acc)
        if acc >= max(val_acc):
            max_index = train_index

    X_train, y_train = X_train[max_index], y_train[max_index]
    model = RVFL(n_node, lam_r, lam_l, w_range, b_range)
    model.train(X_train, y_train, n_class)
    prediction = model.predict(X_test, True)
    acc = model.eval(X_test, y_test)
    print(f'\nTest accuracy: {acc}')

Validation: 1
[[ 5.04374511e-04  2.13875229e-03 -1.17450988e-03 ... -3.45333677e-03
  -6.01979965e-03  5.38097509e-04]
 [ 3.30773902e-03 -3.65858141e-05  1.08044377e-03 ... -4.21977981e-03
   4.92154674e-03  1.13758241e-03]
 [ 7.72101021e-04 -1.81352010e-03  2.05893643e-03 ...  5.73386555e-03
  -5.50368099e-03 -6.33692566e-04]
 ...
 [ 2.31899840e-02  3.30697065e-02  3.12812553e-03 ...  1.21468185e-03
  -7.00948511e-03 -2.54379322e-03]
 [ 2.20132182e-02  3.41635118e-02  3.01462741e-03 ...  7.95530117e-05
  -6.24607254e-03 -2.19576766e-03]
 [ 2.27261934e-02  1.08032377e-02 -1.23046309e-03 ...  4.54572170e-02
   4.71208353e-02  3.07393173e-02]]
Validation accuracy: 0.9310344827586207
Validation: 2
[[ 0.0017806   0.00181141  0.00190106 ... -0.00593323  0.00098155
   0.00136527]
 [ 0.00343268 -0.00120063  0.0018041  ... -0.00026207  0.00017826
   0.00028931]
 [-0.00021828  0.00025807 -0.0003833  ... -0.00393594  0.00258719
   0.00100095]
 ...
 [ 0.01959066  0.0274224  -0.00335025 ...  0.005

In [29]:
# g50c dataset
if __name__=="__main__":
    dataset = loadmat('g50c.mat')
    label = np.array([dataset['y'][i][0] for i in range(len(dataset['y']))])
    data = dataset['X']
    n_class = len(np.unique(label))

    # train-test-split
    X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42)
    kf = KFold(n_splits=10, shuffle=True, random_state=0)
    val_acc = []
    max_index = -1
    
    for i, kf_values in enumerate(kf.split(X_train, y_train)):
    #     print(f'train: {train_index}, val: {val_index}')
        print('Validation: {}'.format(i + 1))
        train_index, val_index = kf_values
        X_val_train, X_val_test = X_train[train_index], X_train[val_index]
        y_val_train, y_val_test = y_train[train_index], y_train[val_index]
        model = RVFL(n_node, lam, w_range, b_range)
        model.train(X_val_train, y_val_train, n_class)
        prediction = model.predict(X_val_test, True)
        acc = model.eval(X_val_test, y_val_test)
        print(f'Validation accuracy: {acc}')
        val_acc.append(acc)
        if acc >= max(val_acc):
            max_index = train_index

    X_train, y_train = X_train[max_index], y_train[max_index]
    model = RVFL(n_node, lam, w_range, b_range)
    model.train(X_train, y_train, n_class)
    prediction = model.predict(X_test, True)
    acc = model.eval(X_test, y_test)
    print(f'\nTest accuracy: {acc}')

Validation: 1
Validation accuracy: 0.4318181818181818
Validation: 2
Validation accuracy: 0.4772727272727273
Validation: 3
Validation accuracy: 0.5454545454545454
Validation: 4
Validation accuracy: 0.5
Validation: 5
Validation accuracy: 0.29545454545454547
Validation: 6
Validation accuracy: 0.5909090909090909
Validation: 7
Validation accuracy: 0.4318181818181818
Validation: 8
Validation accuracy: 0.5227272727272727
Validation: 9
Validation accuracy: 0.5454545454545454
Validation: 10
Validation accuracy: 0.5681818181818182
Accuracy: 0.5363636363636364


In [22]:
# uspst dataset
if __name__=="__main__":
    dataset = loadmat('uspst.mat')
    label = np.array([dataset['y'][i][0] for i in range(len(dataset['y']))])
    data = dataset['X']
    n_class = len(np.unique(label))

    # train-test-split
    X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42)
    kf = KFold(n_splits=10, shuffle=True, random_state=0)
    val_acc = []
    max_index = -1
    
    for i, kf_values in enumerate(kf.split(X_train, y_train)):
    #     print(f'train: {train_index}, val: {val_index}')
        print('Validation: {}'.format(i + 1))
        train_index, val_index = kf_values
        X_val_train, X_val_test = X_train[train_index], X_train[val_index]
        y_val_train, y_val_test = y_train[train_index], y_train[val_index]
        model = RVFL(n_node, lam, w_range, b_range)
        model.train(X_val_train, y_val_train, n_class)
        prediction = model.predict(X_val_test, True)
        acc = model.eval(X_val_test, y_val_test)
        print(f'Validation accuracy: {acc}')
        val_acc.append(acc)
        if acc >= max(val_acc):
            max_index = train_index

    X_train, y_train = X_train[max_index], y_train[max_index]
    model = RVFL(n_node, lam, w_range, b_range)
    model.train(X_train, y_train, n_class)
    prediction = model.predict(X_test, True)
    acc = model.eval(X_test, y_test)
    print(f'\nTest accuracy: {acc}')

Validation: 1
Validation accuracy: 0.8633540372670807
Validation: 2
Validation accuracy: 0.8509316770186336
Validation: 3
Validation accuracy: 0.8757763975155279
Validation: 4
Validation accuracy: 0.8944099378881988
Validation: 5
Validation accuracy: 0.9130434782608695
Validation: 6
Validation accuracy: 0.90625
Validation: 7
Validation accuracy: 0.81875
Validation: 8
Validation accuracy: 0.89375
Validation: 9
Validation accuracy: 0.91875
Validation: 10
Validation accuracy: 0.9125
Accuracy: 0.8955223880597015
